In [17]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
# import joptimise
using Printf
using JSON
using CSV
using DataFrames


In [18]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module
param3b = SailorMoon.dynamics_parameters()
plotly()

C:\Users\yujit\OneDrive\ドキュメント\github\SailorMoon.jl\tests_yuji
C:\Users\yujit\OneDrive\ドキュメント\github\SailorMoon.jl\src


┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240


Plots.PlotlyBackend()

In [19]:
function plot_circle(radius, x, y, n=100)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [20]:
moon     = plot_circle(1-param3b.mu2, param3b.as , 0.0)
earth    = plot_circle(param3b.mu2, param3b.as, 0.0)
earth_ub = plot_circle(param3b.mu2+6375/param3b.lstar, param3b.as, 0.0)
earth_lb = plot_circle(param3b.mu2-6375/param3b.lstar, param3b.as, 0.0)
moon_soi_outer = plot_circle(1-param3b.mu2+66000/param3b.lstar, param3b.as, 0.0);

In [21]:
### PARAMETERS ###################################
# csv file to load the initial solution
filename = "grid_search.csv"
# dv_dir function corresponding to the csv file 
dir_func = SailorMoon.dv_no_thrust 

# 3body parameter
param3b = SailorMoon.dynamics_parameters()

n_arc = 5
##################################################

if dir_func == SailorMoon.dv_no_thrust
    τ_ig = 0.0
else 
    τ_ig = 1.0
end

# load initial guess
df = DataFrame(CSV.File(filename))

# maybe want to use "for row in eachrow(df)" to automate the process...? 
row = df[10,:]

ig_x, lx, ux = SailorMoon.make_ig_bounds(row, τ_ig, n_arc)

([0.017102187919246038, 5.3288707408686875, -0.748534340244752, 1.0, 5.195511758223204, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.017089093332320674, 0.0, -3.141592653589793, 0.8648668816654901, 3.6368582307562427, 0.0, -3.141592653589793, -3.141592653589793, 0.0, -3.141592653589793  …  -3.141592653589793, 0.0, -3.141592653589793, -3.141592653589793, 0.0, -3.141592653589793, -3.141592653589793, 0.0, -3.141592653589793, -3.141592653589793], [0.018128733991321172, 10.0, 3.141592653589793, 1.482628939997983, 6.754165285690165, 1.0, 3.141592653589793, 3.141592653589793, 1.0, 3.141592653589793  …  3.141592653589793, 1.0, 3.141592653589793, 3.141592653589793, 1.0, 3.141592653589793, 3.141592653589793, 1.0, 3.141592653589793, 3.141592653589793])

In [22]:
sv_mid_cart = [row.x_ra, row.y_ra, row.z_ra, row.xdot_ra, row.ydot_ra, row.zdot_ra]
# change the coordinates into cylindrical (only position)
svm_mid_cyl = vcat(SailorMoon.cart2cylind_only_pos(sv_mid_cart), row.m_ra)

tof_leo2mid = row.dt2
tof_mid2lpo = row.dt1
tof = row.tof
rp = row.rp_kep
ra = row.ra_kep
α = row.alpha
m_rp = row.m_rp

θsf = row.thetaf
ϕ0  = row.phi0

2.722713633111154

In [23]:
@time res, sol_param_list, sols_ballistic, _ = SailorMoon.multishoot_trajectory(ig_x, dir_func, n_arc, true, true);
println(res)
norm(res)

SMA [km]: 2.056854118778891e6
ra  [km]: 6580.038120358991
Energy in inertial frame: -0.0022728483236780783
sv0_sb1: [7013.863851873391, -8606.910301905602, 0.0][7.479006307422126, 8.051790331639648, 0.0]
svf_sunb1: 
[389.9280740583398, -0.3177052988902724, -6.7328144038098995e-21, 0.22890158715440737, 1.0054614251591665, 2.3627781548612e-20, 1.235524116664986]
  8.484266 seconds (15.70 M allocations: 1.084 GiB, 7.23% gc time, 12.49% compilation time)
[-0.1828048010738712, 0.7679557263144923, 2.6406431653292557e-20, -0.03838812066377374, 0.26659414171926477, 3.418556837188038e-21, 0.13651342778700526, 0.02162524966439605, 0.10482534446743141, -7.08556844347428e-20, 0.012746787220316913, 0.019906812253334127, 1.2986581379926824e-20, 0.0990106888779807]


0.8580055218846483

In [24]:
arcs_color = Dict(
    "leo_arc" => :navy, 
    "mid_bck_arc" => :red1, 
    "mid_fwd_arc" => :darkorange, 
    "lpo_arc" => :green
)

pcart = plot(
    size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4, legend=false
)
plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, earth_ub[1,:], earth_ub[2,:], c=:black, lw=1.0, label="earth")
plot!(pcart, earth_lb[1,:], earth_lb[2,:], c=:black, lw=1.0, label="earth")

plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

# plot!(hcat(sol_.u...)[1,:], hcat(sol_.u...)[2,:], color=:black, label="backward(from LLO):0.001", set_aspect=:equal, size=(700,700))


# ballistic legs
for sol_ballistic in sols_ballistic
    plot!(pcart, hcat(sol_ballistic.u...)[1,:], hcat(sol_ballistic.u...)[2,:], c=:dodgerblue, label=false)
end

# trajectory
# for i = 1:length(sol_param_list)
#     sol, _, name = sol_param_list[i]
#     #if name == "mid_bck_arc" || name == "mid_fwd_arc" || name == "lpo_arc"
#     plot!(pcart, sol[1,:], sol[2,:], 
#         linewidth=1.5, label="$name", c=arcs_color[name])
#     #end
# end

# trajectory
# starting from 2, excluding the first LEO arc now...
for i = n_arc+1:length(sol_param_list)
    sol, _, name = sol_param_list[i]
    #if name == "mid_bck_arc" || name == "mid_fwd_arc" || name == "lpo_arc"
    plot!(pcart, sol[1,:], sol[2,:], 
        label="$name", c=arcs_color[name])
    scatter!(pcart, [sol[1,1],sol[1,end]], [sol[2,1],sol[2,end]], 
        label=false, c=arcs_color[name], marker=:circle)
    #end
end

# control node
# scatter!(pcart, [x_mid[1]], [x_mid[2]], marker=:circle, color=:black, label="MP")
scatter!(pcart, [param3b.as], [0.0], marker=:circle, color=:black)

# plot the earth when the initial guess is terminated 
θm0 = π - (θsf - param3b.oms * tof)
e0x = param3b.as - param3b.mu2*cos(θm0)
e0y = - param3b.mu2 * sin(θm0)
earth0 = plot_circle(6375/param3b.lstar, e0x, e0y)

plot!(pcart, earth0[1,:], earth0[2,:], c=:red, lw=1.0, label="earth")
scatter!(pcart, [e0x], [e0y], color=:red, markersize=2.5)


pcart

In [25]:
x_LEO, x_mid, x_LPO, tofs, θs = SailorMoon.unpack_x(ig_x, n_arc)
sv0_LEO, θ0_leo, sol_ballistic_fwd = SailorMoon.get_LEO_state(x_LEO, θs, false)

θ0_leo
println("sv0_LEO: ", sv0_LEO) 

sv0_LEO: [388.55631085624094, 0.49615419618341516, 0.0, -1.0136924730453765, 1.3160990646339896, 0.0, 1.0]


In [26]:
# params = [
#     param3b.mu2, param3b.mus, param3b.as, pi-θ0_leo, param3b.oml, param3b.omb, 0.0, 0.0, 0.0, 0.0, 0.0, dir_func
# ]

# s0 = [388.839478293108, -0.02237942003634170,0.0, 7.321143517136880, 7.8456645421150800, 0.00000, 1.0]
# prob = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame2!, sv0_LEO, [0,tofs[4]], params, method=Tsit5(), reltol=1e-12, abstol=1e-12)
# sol = solve(prob)

# plot!(pcart, sol[1,:], sol[2,:], linewidth=1.5)

In [27]:
# scatter!(pcart,[388.3077264],[0.840842418])

In [28]:
# propagate from the lunar radius... 
s0 = [388.3077264, 0.840842418, 1.77E-21,-0.760995653,1.004541054,8.06E-21, 1.0]
t = 24.19563164
θslr = θsf - t * param3b.oms

25.736830542736705

In [29]:
params = [
    param3b.mu2, param3b.mus, param3b.as, pi-θslr, param3b.oml, param3b.omb, 0.0, 0.0, 0.0, 0.0, 0.0, dir_func
]
prob = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame2!, s0, [0,tof_leo2mid/2 + t - tof], params, method=Tsit5(), reltol=1e-12, abstol=1e-12)
sol = solve(prob);

Unrecognized keyword arguments: [:method]

┌ Warning: Unrecognized keyword arguments found. Future versions will error.
│ The only allowed keyword arguments to `solve` are:
│ (:dense, :saveat, :save_idxs, :tstops, :tspan, :d_discontinuities, :save_everystep, :save_on, :save_start, :save_end, :initialize_save, :adaptive, :abstol, :reltol, :dt, :dtmax, :dtmin, :force_dtmin, :internalnorm, :controller, :gamma, :beta1, :beta2, :qmax, :qmin, :qsteady_min, :qsteady_max, :qoldinit, :failfactor, :calck, :alias_u0, :maxiters, :callback, :isoutofdomain, :unstable_check, :verbose, :merge_callbacks, :progress, :progress_steps, :progress_name, :progress_message, :timeseries_errors, :dense_errors, :weak_timeseries_errors, :weak_dense_errors, :wrap, :calculate_error, :initializealg, :alg, :save_noise, :delta, :seed, :alg_hints, :kwargshandle, :trajectories, :batch_size, :sensealg, :advance_to_tstop, :stop_at_next_tstop, :default_set, :second_time, :prob_choice, :alias_jump, :alias_noise)
│ 
│ See https://diffeq.sciml.ai/stable/basics/common_s

In [30]:
plot!(pcart, sol[1,:], sol[2,:], linewidth=1.5, c=:purple)
scatter!(pcart, [sol[1,1], sol[1,end]], [sol[2,1], sol[2,end]], c=:purple)

In [31]:
# # forward propagation from the LEO 
# param3b.mu2, param3b.mus, param3b.as, pi-θ0_leo, param3b.oml, param3b.omb, 0.0, 0.0, 0.0, 0.0, 0.0, dir_func
# ]

# s0 = [388.839478293108, -0.02237942003634170,0.0, 7.321143517136880, 7.8456645421150800, 0.00000, 1.0]
# prob = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame2!, sv0_LEO, [0,tof-t], params, method=Tsit5(), reltol=1e-12, abstol=1e-12)
# sol = solve(prob)

# plot!(pcart, sol[1,:], sol[2,:], linewidth=1.5,c=:pink)
# scatter!(pcart, [sol[1,1], sol[1,end]], [sol[2,1], sol[2,end]], c=:pink)

In [33]:
# backward from the x_lr
params = [
    param3b.mu2, param3b.mus, param3b.as, pi-θslr, param3b.oml, param3b.omb, 0.0, 0.0, 0.0, 0.0, 0.0, dir_func
]
prob = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame2!, s0, [0, t - tof], params, method=Tsit5(), reltol=1e-12, abstol=1e-12)
sol = solve(prob);
plot!(pcart, sol[1,:], sol[2,:], linewidth=1.5, c=:pink)
scatter!(pcart, [sol[1,1], sol[1,end]], [sol[2,1], sol[2,end]], c=:pink)

Unrecognized keyword arguments: [:method]



┌ Warning: Unrecognized keyword arguments found. Future versions will error.
│ The only allowed keyword arguments to `solve` are:
│ (:dense, :saveat, :save_idxs, :tstops, :tspan, :d_discontinuities, :save_everystep, :save_on, :save_start, :save_end, :initialize_save, :adaptive, :abstol, :reltol, :dt, :dtmax, :dtmin, :force_dtmin, :internalnorm, :controller, :gamma, :beta1, :beta2, :qmax, :qmin, :qsteady_min, :qsteady_max, :qoldinit, :failfactor, :calck, :alias_u0, :maxiters, :callback, :isoutofdomain, :unstable_check, :verbose, :merge_callbacks, :progress, :progress_steps, :progress_name, :progress_message, :timeseries_errors, :dense_errors, :weak_timeseries_errors, :weak_dense_errors, :wrap, :calculate_error, :initializealg, :alg, :save_noise, :delta, :seed, :alg_hints, :kwargshandle, :trajectories, :batch_size, :sensealg, :advance_to_tstop, :stop_at_next_tstop, :default_set, :second_time, :prob_choice, :alias_jump, :alias_noise)
│ 
│ See https://diffeq.sciml.ai/stable/basics/common_s